### Define Zones And Scales


In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
import pickle
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from lesson_functions_udacity import *
from sklearn.cross_validation import train_test_split
from scipy.ndimage.measurements import label



C:\Miniconda3\envs\carnd\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:

zone_list = [((700,340),(1280,720),0.61,0.5),
             ((700,370),(1220,530),0.63,0.8),
             ((700,395),(1100,470),0.65,0.5)]
             
             
scales = [[(160,160),(110,110)],
          [(110,110),(75,75)],
          [(75,75),(45,45)]]
          

n = 20
t = 9


### Video Pipeline

In [3]:
frames = []
vehicles = []

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        
        # average over consecutive frames
        if len(vehicles) == labels[1]:
            vehicles[car_number-1].append(np.array(bbox))
            if len(vehicles[car_number-1]) > 35:
                del(vehicles[car_number-1][0])
            bbox = np.int_(np.mean(np.array(vehicles[car_number-1]),axis = 0))     
            bbox = tuple(map(tuple, bbox))
        else:
            if car_number == 1:
                del(vehicles[:])
            vehicles.append([np.array(bbox)])
        
        
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 4)
    # Return the image
    return img


def process_video(image):


    all_windows = []
    for zone, window_list in zip(zone_list,scales):
        zone_windows = []
        thresh = zone[-1]
        overlap = zone[2]
        for window in window_list:
            windows = slide_window(image, x_start_stop=[zone[0][0], zone[1][0]], y_start_stop=[zone[0][1], zone[1][1]], 
                                   xy_window=window, xy_overlap=(overlap, overlap))
            zone_windows.extend(windows)
        all_windows.append([zone_windows,thresh])

    all_windows = all_windows
    
    
    svc = pickle.load(open( "pickle/classifier.p", "rb" ))
    params = pickle.load(open( "pickle/feature_params.p", "rb" ))

    
    X_scaler = params['scaler']
    color_space = params['color_space']
    spatial_size = params['spatial_size']
    hist_bins = params['hist_bins']
    orient = params['orient']
    pix_per_cell = params['pix_per_cell']
    cell_per_block = params['cell_per_block']
    hog_channel = params['hog_channel']
    
    hot_windows = []
    
    for zone_windows in all_windows:
        thresh = zone_windows[-1]
        hot_windows.append(search_windows(image, zone_windows[0], svc, X_scaler, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=True, 
                                hist_feat=True, hog_feat=True, thresh = thresh))
                           
    box_list = []
    for zone in hot_windows:
        for box in zone:
            box_list.append(box)
    
    
    
    frames.append(box_list)
    
    if len(frames) > n:
        del(frames[0])
    
    box_list = []
    for frame in frames:
        box_list.extend(frame)
    
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
    draw_img = None
    heatmap = None
    
    
   
    # Add heat to each box in box list
    heat = add_heat(heat,box_list)

    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,t)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)
    
    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(image), labels)
    
    
    return draw_img
        
    
    

In [4]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML


white_output = 'final_video2.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("project_video.mp4").subclip(40,43)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_video) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video final_video2.mp4
[MoviePy] Writing video final_video2.mp4


100%|███████████████████████████████████▉| 1260/1261 [1:10:01<00:04,  4.39s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: final_video2.mp4 

Wall time: 1h 10min 4s


In [5]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))